In [2]:
import numpy as np
from numpy.linalg import eigh, eig
import pandas as pd
import plotly.express as px
from plotly import graph_objects as go
from scipy.optimize import curve_fit

In [22]:
def lorentz(X, mean, sigma, scale, offset):
    return scale/(np.pi * sigma * (1 + ((X - mean)/sigma)**2)) + offset

def lorentz_diff1(X, mean, sigma, scale, offset):
    return -(2 * sigma * (-mean + X)) / (np.pi * (mean**2 + sigma**2 - 2*mean*X + np.power(X, 2))**2) * scale + offset

def lorentz_diff2(X, mean, sigma, scale, offset):
    return -(2 * sigma * (sigma**2 - 3 * np.power(mean - X, 2))) / (np.pi * np.power(sigma**2 + np.power(mean - X, 2), 3)) * scale + offset

def lorentz_spread(X, mean, sigma, scale, offset):
    return (mean**2 - 2 * mean * X + np.power(X, 2) - sigma**2)/(np.pi * (mean**2 - 2 * mean * X + np.power(X, 2) + sigma**2)**2) * scale + offset

def line(X, offset):
    return np.ones(len(X)) * offset

# def both_lorentz(X, mean1, sigma1, scale1, offset1, mean2, sigma2, scale2, offset2):
#     return lorentz(X, mean1, sigma1, scale1, offset1) + lorentz_shift(X, mean2, sigma2, scale2, offset2)

class Map:
    def __init__(self, src, x, y, **kwargs):
        raw_data = np.loadtxt(src, delimiter='\t')
        if 'begin' in kwargs and 'end' in kwargs:
            raw_data = raw_data[np.r_[0:1, (kwargs['begin']+2):(kwargs['end']+2)]]
        self.shape = (x, y)
        self.raman_shift = raw_data.T[0][2:]
        self.data_matrix = raw_data[2:].T[1:]
        self.data = pd.DataFrame(raw_data[2:], columns = ['RamanShift'] + list(np.arange(0, x*y, 1)))
        self.map = np.array([raw_data[0][1:], raw_data[1][1:]]).T

        self.cov_matrix = np.cov(self.data_matrix)
        
        eigenvalues, eigenvectors = eigh(self.cov_matrix)
        idx = np.flip(eigenvalues.argsort())
        self.weights = eigenvalues[idx] / sum(eigenvalues)
        self.vectors = eigenvectors.T[idx]
        
        self.base_vectors = np.dot(self.vectors, self.data_matrix)
    
    def reconstruct(self, count):
        return np.dot(self.vectors[:count].T, self.base_vectors[:count])
    
    def plotComponent(self, num):
        fig = px.imshow(np.reshape(self.vectors[num], self.shape), title ='Komponent ' + str(num))
        return fig
    
    def plotPhysicalComponent(self, v):
        img = np.dot(np.array(v), self.vectors[:len(v)])
        fig = px.imshow(np.reshape(img, self.shape), title ='Peak')
        return fig
    
    def plotCovMatrix(self):
        fig = px.imshow(self.cov_matrix, title = 'Macierz kowariancji')
        return fig
    
    def plotWeights(self, cum, log):
        if cum:
            fig = px.bar(np.cumsum(self.weights), title = 'Wagi', log_y=log)
        else:
            fig = px.bar(self.weights, title = 'Wagi', log_y=log)
        return fig
    
    def plotSpectrums(self, ids, **kwargs):
        fig = go.Figure()
        if 'base' not in kwargs:
            for num in ids:
                fig.add_scatter(x = self.raman_shift, y = self.data_matrix[num], mode='lines', name = 'ID: ' + str(num))
        else:
            r = self.reconstruct(kwargs['base'])
            for num in ids:
                fig.add_scatter(x = self.raman_shift, y = r[num], mode='lines', name = 'ID: ' + str(num))
        fig.update_layout(hovermode="x")
        return fig
    
    def plotSpectrum(self, num, **kwargs):
        return self.plotSpectrums([num], **kwargs)
    
    def plotBaseVectors(self, count):
        fig = go.Figure()
        for num in range(count):
            fig.add_scatter(x = self.raman_shift, y = self.base_vectors[num], mode='lines', name = 'Num: ' + str(num))
        fig.update_layout(hovermode="x")
        return fig
            
    def locToId(self, x, y):
        return np.argmin(list(map(lambda e: np.linalg.norm([x, y] - e), self.map)))
    
    def posToId(self, x, y):
        return self.shape[0] * y + x
    
    def draw_curve(self, curve, params, a, b, **draw):
        fig = go.Figure()
        if 'spectrum' in draw:
            fig.add_scatter(x = self.raman_shift, y = self.data_matrix[draw['spectrum']], mode='lines')
        if 'base_vector' in draw:
            fig.add_scatter(x = self.raman_shift, y = self.base_vectors[draw['base_vector']], mode='lines')
        data_slice = self.data[(self.data['RamanShift'] >= a) & (self.data['RamanShift'] <= b)]['RamanShift']
        fig.add_scatter(mode='lines', x = data_slice, y = curve(data_slice, *params))
        fig.update_layout(hovermode="x")
        return fig
    
    def fit_curve(self, curve, p0, a, b, seek, **plot):
        y = 0
        if 'spectrum' in plot:
            y = self.data_matrix[plot['spectrum'], (self.data['RamanShift'] >= a) & (self.data['RamanShift'] <= b)]
        if 'base_vector' in plot:
            y = self.base_vectors[plot['base_vector'], (self.data['RamanShift'] >= a) & (self.data['RamanShift'] <= b)]
        data_slice = self.data[(self.data['RamanShift'] >= a) & (self.data['RamanShift'] <= b)]['RamanShift']
        
        bounds = 0
        if 'bounds' in plot and plot['bounds'] == 'inf':
            bounds = ((-np.inf, -np.inf, -np.inf, -np.inf), (np.inf, np.inf, np.inf, np.inf))
        else:
            bounds = ((p0[0]-2, -np.inf, -np.inf, -np.inf), (p0[0]+2, np.inf, np.inf, np.inf))
        
        param, cov = curve_fit(curve, data_slice, y, p0=p0, bounds = bounds)
        print(param)
        if 'draw' in plot and plot['draw'] == True:
            self.draw_curve(curve, param, a, b, **plot).show()
        return param[seek]

In [23]:
map_a = Map('../data/GA25_10B_532nm_100%_1x3sec_x100_xc1200_A_trojkat1_mapa_10x10um_step_0.3um_data.txt', 34, 34, begin=250, end=450)

In [ ]:
map_a.plotWeights(False, True).show()

In [ ]:
map_a.plotBaseVectors(10).show()

In [48]:
## Map A
t_matrix = np.zeros((5,5))

draw = False
# for i in range(6):
#     t_matrix[0][i] = np.average(map_a.base_vectors[i], weights=((map_a.raman_shift < 1310) | (map_a.raman_shift > 1420)) )
    # t_matrix[0][i] = map_a.draw_curve(line, [np.average(map_a.base_vectors[i], weights=((map_a.raman_shift < 1310) | (map_a.raman_shift > 1420)) )], 1200, 1500, base_vector = i, draw = True, bounds = 'inf').show()

t_matrix[0][0] = map_a.fit_curve(lorentz, (1392, 2.5, -17 * 10**5, -7000), 1380, 1400, 2, base_vector = 0, draw = draw)

t_matrix[1][0] = map_a.fit_curve(lorentz, (1345, 14,  -5 * 10**5, -10000), 1330, 1360, 2, base_vector = 0, draw = draw)
t_matrix[1][3] = map_a.fit_curve(lorentz, (1345, 16,  3.2 * 10**5, -4000), 1330, 1360, 2, base_vector = 3, draw = draw)

t_matrix[2][1] = map_a.fit_curve(lorentz, (1369, 10,  0.8 * 10**5,  1000), 1355, 1380, 2, base_vector = 1, draw = draw)
t_matrix[2][2] = map_a.fit_curve(lorentz, (1369, 10, -2.4 * 10**5, -1400), 1355, 1380, 2, base_vector = 2, draw = draw)

t_matrix[3][1] = map_a.fit_curve(lorentz_diff1, (1392, 2.5, -17 * 10**5, -7000), 1385, 1400, 2, base_vector = 1, draw = draw, bounds = 'inf')
t_matrix[3][2] = map_a.fit_curve(lorentz_diff1, (1392, 2.5, -17 * 10**5, -7000), 1385, 1400, 2, base_vector = 2, draw = draw, bounds = 'inf')
t_matrix[3][3] = map_a.fit_curve(lorentz_diff1, (1392, 2.5, -17 * 10**5, -7000), 1385, 1400, 2, base_vector = 3, draw = draw, bounds = 'inf')

# t_matrix[4][2] = map_a.fit_curve(lorentz_diff2, [ 1.39e+03, 3e+00, 10.8e+04, -1.4e+03], 1380, 1400, 2, base_vector = 2, draw = True, bounds = 'inf')
# t_matrix[4][3] = map_a.fit_curve(lorentz_diff2, [ 1.39e+03, 3e+00, 10.8e+04, -1.4e+03], 1380, 1400, 2, base_vector = 3, draw = True, bounds = 'inf')
t_matrix[4][4] = map_a.fit_curve(lorentz_diff2, [ 1.39e+03, 3e+00, 10.8e+04, -1.4e+03], 1380, 1400, 2, base_vector = 4, draw = True, bounds = 'inf')

t_matrix

[ 1.39214541e+03  2.43557140e+00 -1.65031680e+06 -2.88549932e+03]
[ 1.34522635e+03  1.55279248e+01 -6.45129936e+05 -7.53953211e+03]
[ 1.34445874e+03  1.77266388e+01  3.67512771e+05 -4.06361544e+03]
[1.36866873e+03 8.54118216e+00 6.16217337e+04 1.18293094e+03]
[ 1.36843431e+03  9.97630199e+00 -2.40442368e+05 -1.57457061e+03]
[1.39190191e+03 2.72960605e+00 6.74604102e+05 2.07338306e+03]
[ 1.39278530e+03  2.20727076e+00  8.02567135e+04 -2.72316944e+03]
[ 1.39433161e+03  2.57365113e+00 -7.01780502e+04 -1.71260026e+03]
[ 1.39061238e+03  4.62607352e+00 -6.55782081e+05 -3.26815474e+03]


[ 1.39258299e+03  2.99088173e+00  1.19681247e+05 -1.89567155e+03]


[1.39201806e+03 2.91450205e+00 3.29325215e+05 2.02458738e+03]


array([[-1650316.79563961,        0.        ,        0.        ,
               0.        ,        0.        ],
       [ -645129.93611166,        0.        ,        0.        ,
          367512.77129684,        0.        ],
       [       0.        ,    61621.73369661,  -240442.3677895 ,
               0.        ,        0.        ],
       [       0.        ,   674604.10153941,    80256.71349783,
          -70178.05019162,        0.        ],
       [       0.        ,        0.        ,  -655782.08061975,
          119681.24683707,   329325.21450672]])

In [52]:
map_a.draw_curve(lorentz_diff2, [ 1.39113772e+03, 3e+00, 10.79995135e+04, -1.38837064e+03], 1380, 1400, base_vector = 4, draw = draw, bounds = 'inf').write_html('test.html')


In [49]:
new_base_vectors = np.dot(map_a.base_vectors[:5].T, np.linalg.inv(t_matrix)).T
new_vectors = np.dot(t_matrix, map_a.vectors[:5])

px.line(x = map_a.raman_shift, y = new_base_vectors[0]).show()
# px.imshow(np.reshape(new_vectors[0], map_a.shape)).show()
px.line(x = map_a.raman_shift, y = new_base_vectors[1]).show()
# px.imshow(np.reshape(new_vectors[1], map_a.shape)).show()
px.line(x = map_a.raman_shift, y = new_base_vectors[2]).show()
# px.imshow(np.reshape(new_vectors[2], map_a.shape)).show()
px.line(x = map_a.raman_shift, y = new_base_vectors[3]).show()
# px.imshow(np.reshape(new_vectors[3], map_a.shape)).show()
px.line(x = map_a.raman_shift, y = new_base_vectors[4]).show()
# px.imshow(np.reshape(new_vectors[4], map_a.shape)).show()
# px.line(x = map_a.raman_shift, y = new_base_vectors[5]).show()
# px.imshow(np.reshape(new_vectors[5], map_a.shape)).show()

In [ ]:
map_a.plotSpectrum(map_a.posToId(11, 10))